In [1]:
import pandas as pd

from pandas import set_option
set_option("display.max_rows", 20)

LARGE_FIGSIZE = (12, 8)

In [2]:
cd data/

/Users/esteban/unm/MachineLearning/Project1/data


In [3]:
ls

play_tennis.txt                           validation-classifications-pre-prune.txt
testing.txt                               validation.txt
training.txt


In [4]:
training = pd.read_table("training.txt", sep=",", 
                         names=["label","cap-shape","cap-surface","cap-color","bruises","oder",
                                "gill-attachment","gill-spacing","gill-size","gill-color","stalk-shape",
                                "stalk-root","stalk-surface-above-ring","stalk-surface-below-ring",
                                "stalk-color-above-ring","stalk-color-below-ring","veil-type",
                                "veil-color","ring-number","ring-type","spore-print-color",
                                "population","habitat"])
training

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,y,e,f,f,f,c,n,b,...,s,p,w,p,w,o,e,w,v,l
1,e,x,f,g,f,n,f,c,n,n,...,s,w,w,p,w,o,p,k,v,u
2,p,f,s,n,t,p,f,c,n,p,...,s,w,w,p,w,o,p,n,s,u
3,p,x,y,g,f,f,f,c,b,g,...,k,b,n,p,w,o,l,h,y,p
4,e,x,y,g,t,n,f,c,b,n,...,s,p,g,p,w,o,p,k,y,d
5,e,x,f,w,t,a,f,w,n,n,...,s,w,w,p,w,o,p,n,v,d
6,e,b,f,g,f,n,f,w,b,p,...,k,w,w,p,w,t,p,w,s,g
7,e,x,f,n,t,n,f,c,b,n,...,s,p,p,p,w,o,p,n,v,d
8,e,k,f,c,f,n,f,w,n,w,...,s,w,n,p,w,o,e,w,v,l
9,e,s,f,g,f,n,f,c,n,k,...,s,w,w,p,w,o,p,k,v,u


In [50]:
import math

#represents a test node in the tree
class Node:
    decision_attribute = ""
    
    
    def __init__(self):
        self.decision_attribute = ""
        self.branches = {}
        
    def add_branch(self,attribute_value,node):
        self.branches[attribute_value] = node
    
    def get_branches(self):
        return self.branches

#represents a leaf node in the tree
class Leaf:
    label = ""
    
positive_value = ""
negative_value = ""
evaluation_criteria = ""

#key = dof, values[0] = alpha 0.5, values[1] =  alpha 0.05, values[2] = alpha 0.01
chi_square_look_up = {
                    1: [0.4549,3.841,6.635],
                    2: [1.3863,5.991,9.21],
                    3: [2.366,7.815,11.345],
                    4: [3.3567,9.488,13.277],
                    5: [4.3515,11.07,15.086],
                    6: [5.3481,12.592,16.812],
                    7: [6.3458,14.067,18.475],
                    8: [7.3441,15.507,20.09],
                    9: [8.3428,16.919,21.666],
                    10: [9.3418,18.307,23.209],
                    11: [10.341,19.675,24.725],
                    12: [11.3403,21.026,26.217],
                    13: [12.3398,22.362,27.688],
                    14: [13.3393,23.685,29.141],
                    15: [14.3389,24.996,30.578],
                    16: [15.3385,26.296,32],
                    17: [16.3382,27.587,33.409],
                    18: [17.3379,28.869,34.805],
                    19: [18.3377,30.144,36.191],
                    20: [19.3374,31.41,37.566]
                    }

attributes_master = ["cap-shape","cap-surface","cap-color","bruises","oder",
                "gill-attachment","gill-spacing","gill-size","gill-color","stalk-shape",
                "stalk-root","stalk-surface-above-ring","stalk-surface-below-ring",
                "stalk-color-above-ring","stalk-color-below-ring","veil-type",
                "veil-color","ring-number","ring-type","spore-print-color",
                "population","habitat"]

valid_values = {"label": ["p","e"],
                "cap-shape": ["b","c","x","f","k","s"],
                "cap-surface": ["f","g","y","s"],
                "cap-color": ["n","b","c","g","r","p","u","e","w","y"],
                "bruises": ["t","f"],
                "oder": ["a","l","c","y","f","m","n","p","s"],
                "gill-attachment": ["a","d","f","n"],
                "gill-spacing": ["c","w","d"],
                "gill-size": ["b","n"],
                "gill-color": ["k","n","b","h","g","r","o","p","u","e","w","y"],
                "stalk-shape": ["e","t"],
                "stalk-root": ["b","c","u","e","z","r","?"],
                "stalk-surface-above-ring": ["f","y","k","s"],
                "stalk-surface-below-ring": ["f","y","k","s"],
                "stalk-color-above-ring": ["n","b","c","g","o","p","e","w","y"],
                "stalk-color-below-ring": ["n","b","c","g","o","p","e","w","y"],
                "veil-type": ["p","u"],
                "veil-color": ["n","o","w","y"],
                "ring-number": ["n","o","t"],
                "ring-type": ["c","e","f","l","n","p","s","z"],
                "spore-print-color": ["k","n","b","h","r","o","u","w","y"],
                "population": ["a","c","n","s","v","y"],
                "habitat": ["g","l","m","p","u","w","d"]
                }

def degress_of_freedom(attribute):
    return len(valid_values[attribute]) - 1

def chi_square_value_for_attribute(attribute,alpha):
    values = chi_square_look_up[degress_of_freedom(attribute)]
    if alpha == '0.5':
        return values[0]
    elif alpha == '0.05':
        return values[1]
    elif alpha == '0.01':
        return values[2]
    else:
        return 0.0
    

def size(data):
    return len(data.index)

def filter_data(attribute, value):
    return examples[examples[attribute] == value]
    
    
def all_one_value(examples, attribute, attribute_value):
    #test that all examples have the same value, examples are a Pandas Dataframe
    filtered_examples = examples[examples[attribute] == attribute_value]
    return len(filtered_examples.index) == len(examples.index)

def possible_range_of_values(attribute):
    return valid_values[attribute]

def extract_most_common_value(examples, attribute):
    #print("most common for ", attribute)
    values = possible_range_of_values(attribute)
    most_common = ""
    highest_count = 0
    for value in values:
        filtered = examples[examples[attribute] == value]
        if len(filtered.index) > highest_count:
            highest_count = len(filtered.index)
            most_common = value  
    return most_common


def chi_square_calculation(examples, target_attribute,attribute_v):
    chi_square = 0.0
    total = len(examples.index)
    if total == 0:
        return 0.0
    positive_filtered_examples = examples[examples[target_attribute] == positive_value]
    negative_filtered_examples = examples[examples[target_attribute] == negative_value]
    
    observed_positive = len(positive_filtered_examples.index)
    expected_positive = (total/2.0)
                
    observed_negative = len(negative_filtered_examples.index)
    expected_negative = (total/2.0)
    
    pos = ((observed_positive - expected_positive)**2)/expected_positive
    neg = ((observed_negative - expected_negative)**2)/expected_negative
    return pos + neg
          
    
def chi_square_test(examples, target_attribute,attribute):
    chi_square_sum = 0.0
    values = possible_range_of_values(attribute)
    for value in values:
        examples_v = examples[examples[attribute] == value]
        chi_square_v = chi_square_calculation(examples_v,target_attribute,value)
        chi_square_sum = chi_square_sum + chi_square_v
    return chi_square_sum
    

def calculate_entropy(examples,target_attribute):
    entropy = 0.0
    total = len(examples.index)
    positive_filtered_examples = examples[examples[target_attribute] == positive_value]
    negative_filtered_examples = examples[examples[target_attribute] == negative_value]
    
    num_positive = len(positive_filtered_examples.index)
    calc_positive = 0.0
    if num_positive != 0:
        calc_positive = -(num_positive/total)*math.log((num_positive/total),2)
            
        
    num_negative = len(negative_filtered_examples.index)
    calc_negative = 0.0
    if num_negative != 0:
        calc_negative = - (num_negative/total)*math.log((num_negative/total),2)
        
        
    return calc_positive + calc_negative
   

def calculate_misclassification_error(examples, target_attribute):
    error = 0.0
    total = len(examples.index)
    if total == 0:
        return 1.0
    positive_filtered_examples = examples[examples[target_attribute] == positive_value]
    negative_filtered_examples = examples[examples[target_attribute] == negative_value]
    
    num_positive = len(positive_filtered_examples.index)
    p_positive = (num_positive/total)
                
    num_negative = len(negative_filtered_examples.index)
    p_negative = (num_negative/total)
        
        
    
    error = 1.0 - max([p_positive,p_negative])
    return error
    
    
def determine_best_attribute_entropy(examples, target_attribute, attributes):
    best_attribute = ""
    max_information_gain = 0.0
   
    #find the entropy S
    entropy_s = calculate_entropy(examples,target_attribute)
    size_of_s = len(examples.index)
    
    
    #loop through attributes and calculate the gain
    for attribute in attributes:
        values = possible_range_of_values(attribute)
        information_gain = entropy_s
        for value in values:
            examples_v = examples[examples[attribute] == value]
            entropy_v = calculate_entropy(examples_v,target_attribute)
            size_of_examples_v = len(examples_v.index)
            weight = (size_of_examples_v/size_of_s)
            information_gain = information_gain - weight*entropy_v
        if information_gain > max_information_gain:
            max_information_gain = information_gain
            best_attribute = attribute
        
       
    return best_attribute

def determine_best_attribute_misclassification_error(examples, target_attribute, attributes):
    best_attribute = ""
    max_information_gain = 0.0
    #find the entropy S
    error_s = calculate_misclassification_error(examples,target_attribute)
    size_of_s = len(examples.index)
    
    #loop through attributes and calculate the gain
    for attribute in attributes:
        values = possible_range_of_values(attribute)
        information_gain = error_s
        for value in values:
            examples_v = examples[examples[attribute] == value]
            error_v = calculate_misclassification_error(examples_v,target_attribute)
            size_of_examples_v = len(examples_v.index)
            weight = (size_of_examples_v/size_of_s)
            information_gain = information_gain - weight*error_v
        if information_gain > max_information_gain:
            max_information_gain = information_gain
            best_attribute = attribute
       
    return best_attribute

def id3(examples, target_attribute, attribute_list, alpha):
    root = Node()
    if(all_one_value(examples, target_attribute, positive_value)):
        leaf = Leaf()
        leaf.label = positive_value
        return leaf
    if(all_one_value(examples, target_attribute, negative_value)):
        leaf = Leaf()
        leaf.label = negative_value
        return leaf
    if(len(attributes) == 0):
        leaf = Leaf()
        leaf.label = extract_most_common_value(examples,target_attribute)
        return leaf
    if evaluation_criteria == 'entropy':
        a = determine_best_attribute_entropy(examples, target_attribute, attribute_list)
    else:
        a = determine_best_attribute_misclassification_error(examples, target_attribute, attribute_list)
    chi = chi_square_test(examples,target_attribute,a)
    if chi < chi_square_value_for_attribute(a, alpha):
        print("prune: ", a)
        leaf = Leaf()
        leaf.label = extract_most_common_value(examples,target_attribute)
        return leaf
    root.decision_attribute = a
   
    values = possible_range_of_values(a)
    for value in values:
        root.branches[value] = ""
        examples_v = examples[examples[a] == value]
        size_of_examples_v = len(examples_v.index)
        if size_of_examples_v == 0:
            leaf = Leaf()
            leaf.label = extract_most_common_value(examples,target_attribute)
            root.add_branch(value,leaf)
        else:
            if a in attribute_list:
                attribute_list.remove(a)
            node = id3(examples_v,target_attribute,attribute_list, alpha)
            root.add_branch(value, node)
    return root


def split_child(attribute_values, node):
    return node.get_branches()[attribute_values[node.decision_attribute]]

def classify(attribute_values, node):
    if type(node) is Leaf:
        return node.label
    else:
        return classify(attribute_values, split_child(attribute_values,node))
    
    
    

In [52]:
attributes = ["cap-shape","cap-surface","cap-color","bruises","oder",
                "gill-attachment","gill-spacing","gill-size","gill-color","stalk-shape",
                "stalk-root","stalk-surface-above-ring","stalk-surface-below-ring",
                "stalk-color-above-ring","stalk-color-below-ring","veil-type",
                "veil-color","ring-number","ring-type","spore-print-color",
                "population","habitat"]
positive_value = "e"
negative_value = "p"
evaluation_criteria = "misclassification"
alpha = '0.01'
id3_root = id3(training,"label",attributes,alpha)


In [48]:
testing_data = pd.read_table("testing.txt", sep=",", 
                             names=["label","cap-shape","cap-surface","cap-color","bruises","oder",
                            "gill-attachment","gill-spacing","gill-size","gill-color","stalk-shape",
                            "stalk-root","stalk-surface-above-ring","stalk-surface-below-ring",
                            "stalk-color-above-ring","stalk-color-below-ring","veil-type",
                            "veil-color","ring-number","ring-type","spore-print-color",
                            "population","habitat"])
testing_data


,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,e,x,s,n,f,n,a,c,b,o,...,s,o,o,p,n,o,p,y,v,l
1,e,f,y,n,t,n,f,c,b,w,...,s,g,w,p,w,o,p,n,v,d
2,p,f,y,g,f,f,f,c,b,g,...,k,n,n,p,w,o,l,h,v,d
3,e,x,s,w,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,m
4,e,f,y,u,f,n,f,c,n,h,...,f,w,w,p,w,o,f,h,v,d
5,e,k,f,c,f,n,f,w,n,w,...,f,w,n,p,w,o,e,w,v,l
6,e,f,s,w,f,n,f,w,b,h,...,s,w,w,p,w,o,e,k,a,g
7,e,f,y,n,t,n,f,c,b,p,...,s,g,w,p,w,o,p,n,y,d
8,p,f,f,y,f,f,f,c,b,h,...,k,b,b,p,w,o,l,h,y,p
9,p,k,y,n,f,y,f,c,n,b,...,k,p,p,p,w,o,e,w,v,p


In [13]:
validation_data = pd.read_table("validation.txt", sep=",", 
                                names=["label","cap-shape","cap-surface","cap-color","bruises","oder",
                                        "gill-attachment","gill-spacing","gill-size","gill-color","stalk-shape",
                                        "stalk-root","stalk-surface-above-ring","stalk-surface-below-ring",
                                        "stalk-color-above-ring","stalk-color-below-ring","veil-type",
                                        "veil-color","ring-number","ring-type","spore-print-color",
                                        "population","habitat"])

validation_data

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,n,f,f,y,f,f,f,c,b,g,...,k,b,b,p,w,o,l,h,y,g
1,n,f,y,n,f,f,f,c,n,b,...,s,p,p,p,w,o,e,w,v,l
2,n,f,f,g,t,n,f,c,b,u,...,s,w,w,p,w,o,p,n,v,d
3,n,x,y,e,f,y,f,c,n,b,...,s,p,p,p,w,o,e,w,v,d
4,n,f,y,y,f,f,f,c,b,h,...,k,p,n,p,w,o,l,h,y,g
5,n,f,s,e,f,f,f,c,n,b,...,k,w,w,p,w,o,e,w,v,p
6,n,x,y,g,t,n,f,c,b,u,...,s,g,w,p,w,o,p,k,v,d
7,n,f,s,g,t,f,f,c,b,h,...,f,w,w,p,w,o,p,h,s,g
8,n,x,y,y,t,a,f,c,b,w,...,s,w,w,p,w,o,p,k,n,g
9,n,f,f,n,t,n,f,c,b,w,...,s,p,w,p,w,o,p,n,y,d


In [49]:
count_correct = 0
count_false = 0
for i,r in testing_data.iterrows():
    if r["label"] == classify(r,id3_root):
        count_correct = count_correct + 1
    else:
        count_false = count_false + 1
print("correct: ", count_correct," false: ",count_false)
    

correct:  2031  false:  0


In [27]:
count_correct = 0
count_false = 0
file = open("validation-classifications-pre-prune.txt", "w")
for i,r in validation_data.iterrows():
    file.write(classify(r,id3_root) + "\n")

file.close()
    